In [ ]:
!wget https://natura.di.uminho.pt/download/TGZ/Dictionaries/hunspell/LATEST/hunspell-pt_PT-latest.tar.gz

In [ ]:
!tar xzf hunspell-pt_PT-latest.tar.gz

In [1]:
# Leitura do csv com as stop words removidas
import pandas as pd
df = pd.read_csv("comentarios_tratados.csv")
df.head(10)

,text,toxic
0,meu nivel de amizade com isis é ela ter meu in...,1.0
1,o cara adultera dados que foram desmascarados...,1.0
2,o cara só é simplesmente o maior vencedor d...,1.0
3,eu to chorando vei vsf e eu nem staneio izone ...,1.0
4,eleitor do bolsonaro é tão ignorante q não per...,1.0
5,vai responder as outras 75 conversas e para de...,1.0
6,tem um do jack com a msm música e agr não sei ...,0.0
7,mais é ruim pra pedir desafio esse técnico do ...,1.0
8,eu fico vendo isso e penso desvantagens do kp...,1.0
9,frio do caralho parece até q to dentro do teu ...,1.0


In [ ]:
'''
from spellchecker import SpellChecker
spell = SpellChecker(language='pt')
df['text'].apply(lambda x: " ".join([spell.correction(i) for i in x.split()]))
'''

In [ ]:
df["toxic"] = df["homophobia"] + df["obscene"] + df["insult"] + df["racism"] + df["misogyny"] + df["xenophobia"]
df.drop(['obscene','insult','racism','misogyny','xenophobia','homophobia'],axis=1,inplace=True)
df.head()

In [2]:
# Remoção de acentos
import unicodedata
df['text'] = df['text'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII','ignore').decode('ASCII'))
df.head()

,text,toxic
0,meu nivel de amizade com isis e ela ter meu in...,1.0
1,o cara adultera dados que foram desmascarados...,1.0
2,o cara so e simplesmente o maior vencedor d...,1.0
3,eu to chorando vei vsf e eu nem staneio izone ...,1.0
4,eleitor do bolsonaro e tao ignorante q nao per...,1.0


In [3]:
# Cria uma classificação binária para os dados tóxicos e os não tóxicos
df.loc[df['toxic'] == 0, 'toxic'] = 0
df.loc[df['toxic'] > 0, 'toxic'] = 1
df.head()

,text,toxic
0,meu nivel de amizade com isis e ela ter meu in...,1.0
1,o cara adultera dados que foram desmascarados...,1.0
2,o cara so e simplesmente o maior vencedor d...,1.0
3,eu to chorando vei vsf e eu nem staneio izone ...,1.0
4,eleitor do bolsonaro e tao ignorante q nao per...,1.0


In [4]:
!pip install cyhunspell

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import re
def remove_special_characters(tweet):
    tweet = re.sub(r'http\S+', '', tweet)       # remove os links do tweet
    tweet = re.sub(r't.co\S+', '', tweet)       # remove os links 
    tweet = re.sub(r'#\S+', '', tweet)          # remove hashtags
    tweet = re.sub(r"[.]\s+", '', tweet)        # remove reticências
    emojis = re.compile("["                     # remove os emojis
                      u"\U0001F600-\U0001F64F"  
                      u"\U0001F300-\U0001F5FF"  
                      u"\U0001F680-\U0001F6FF"  
                      u"\U0001F1E0-\U0001F1FF"  
                      u"\U00002500-\U00002BEF"  
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)

    tweet = re.sub(emojis, '', tweet)               # remove os emojis
    tweet = tweet.replace('"', "")                  # remove as aspas
    tweet = re.sub("[-*!,$><:.+?=]", '', tweet)     # remove demais caracteres especiais
    tweet = re.sub(r'  ', ' ', tweet)               # remove espaços duplos
    return tweet.lower()

In [6]:
def write_replies(reply):
        tweet_text = remove_special_characters(reply)
        tweet_text = re.sub(r'@\S+', '', tweet_text)
        # Quando houver retweet
        if (tweet_text[0] == 'r'):
            tweet_text = tweet_text[4:]
        return tweet_text

In [7]:
df['text'] = df['text'].apply(write_replies)
df.head()

,text,toxic
0,meu nivel de amizade com isis e ela ter meu in...,1.0
1,o cara adultera dados que foram desmascarados...,1.0
2,o cara so e simplesmente o maior vencedor da...,1.0
3,eu to chorando vei vsf e eu nem staneio izone ...,1.0
4,eleitor do bolsonaro e tao ignorante q nao per...,1.0


In [8]:
# Bruxaria 1
import hunspell

def tokenize(sentence):
    tokens_regex = re.compile(r"([., :;\n()\"!?\/&%+])", flags=re.IGNORECASE)
    tokens = re.split(tokens_regex, sentence)
    postprocess = []
    postprocess_regex = re.compile(r"\b(\w+)-(me|te|se|nos|vos|o|os|a|as|lo|los|la|las|lhe|lhes|lha|lhas|lho|lhos|no|na|nas|mo|ma|mos|mas|to|ta|tos|tas)\b", flags=re.IGNORECASE)
    for token in tokens:
        for token2 in re.split(postprocess_regex, token):
            if token2.strip():
                postprocess.append(token2)

    return postprocess

h = hunspell.Hunspell("pt_PT", hunspell_data_dir="./hunspell-pt_PT-20210608/")

In [9]:
def lemmetization(x):
    tokens = tokenize(x)
    lemmas = []
    for token in tokens:
        lemma = h.stem(token)
        if len(lemma) == 1:
            lemmas.append(lemma[0])
        else:
            lemmas.append(token)

    return " ".join(lemmas)

In [10]:
df['text'] = df['text'].apply(lambda x: lemmetization(x))
df.sample(20)

,text,toxic
13671,mais feio que eu hj so eu ontem,1.0
4314,achar que eu sou a unica pessoa de campo que n...,0.0
4240,quem e esse passa visao que vou mandar passar ele,0.0
16880,kkkkkkkkkk que povo fudido,1.0
6977,yoonminau [nsfw] onde jimin mandar um nude sem...,0.0
6182,os melhores viloes ever passar no sua tl,0.0
11839,o que que tem terca cara de pica,1.0
670,to eu aqui no oficina e os cara do loja do lad...,0.0
15751,eu abel vai se fuder o meu flamengo nao precis...,1.0
16047,putinha aguentar 3,1.0


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/augusto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# Remoção das stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words('portuguese'))

text_treated = []

for reply in df.text:
    word_list = reply.split()

    for word in word_list:
        if(word in stopwords):
            word_list.remove(word)
    word_list = " ".join(word_list)
    text_treated.append(word_list)

i = 0
for row in df.iterrows():
    df.iloc[i, 0] = text_treated[i]
    i = i + 1

In [13]:
df.sample(20)

,text,toxic
10171,so feio dms,0.0
1866,vzs me botar lugar meu vizinho tipo ficar puto...,1.0
14416,vai fuder amigo,1.0
2157,evidente canalha petista culpar pleonasmo voce...,1.0
10023,adorar noit perceber q n mt amigo proximo d fa...,1.0
7290,ces comecaram mamar cara agora virar criatura ...,1.0
17475,camarote comunista ralar no globo lixo,1.0
12899,julieta que contar sobre barbar aaaa pqp,0.0
18370,porra sei,0.0
19485,povo nao precisa stfo povo nao escolher ningue...,0.0


In [14]:
# Remove as colunas caso ocorra algum NaN
df.dropna(inplace=True)
df.describe()

,toxic
count,21000.000000
mean,0.440714
std,0.496485
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [18]:
# Define qual coluna cada variável receberá
x = df.text
y = df.toxic

In [19]:
x.values.shape

(21000,)

In [22]:
# Para fins de colocar no streamlit
from joblib import dump
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3000)
x_vectorized = vectorizer.fit_transform(x.values)
vectorizer_fitted = vectorizer.fit(x.values)
dump(vectorizer_fitted, '../visualization/models/Vectorizer_NaiveBayes_MultinomialNB.joblib')

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier_fitted = classifier.fit(x_vectorized, y)
dump(classifier_fitted, '../visualization/models/NaiveBayes_MultinomialNB.joblib') 

['../visualization/models/NaiveBayes_MultinomialNB.joblib']

In [20]:
# Define quais serão os dados de treino e de teste, já com o x vetorizado
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [21]:
x_train

2519     no final entender porra todo me acertar a gale...
1855                          vitao perfeito 0 defeito pqp
11805    carinha nervoso ( mongoloide q nao sabe quer i...
7440     achar este pesquisa nao fidedigno dever ser me...
20349    luhan vc vai msm forcar ficar colocar foto de ...
                               ...                        
4910     cs fala cara desperdiou cara depresso perder p...
19360    premiacoes coreano so tornar irrelevante parti...
7614     hahahahahahahahahahahahahahahah conseguir ouvi...
10403    seu nocao caralho voce acha o problema seu nao...
2836                               ridicula kkkkkkkkkkkkkk
Name: text, Length: 16800, dtype: object

In [20]:
# O CountVectorizer conta quantas vezes cada palavra aparece em cada e-mail
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=3000)
x_vectorized = vectorizer.fit_transform(x_train.values)

In [55]:
vectorizer_fitted = vectorizer.fit(x_train.values)

In [50]:
# Instanciando o classificador Naive-Bayes
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
targets = y_train.values

# Utilização da validação cruzada
from sklearn.model_selection import cross_validate
scores = cross_validate(classifier, x_vectorized, y_train, cv=10, scoring=('f1', 'accuracy','precision'), return_train_score=True)

In [54]:
classifier_fitted = classifier.fit(x_vectorized, y_train)

In [ ]:
classifier_fitted.predict(x_train[0])

In [51]:
scores['test_f1'].mean()

0.6790890571058457

In [52]:
scores['test_accuracy'].mean()

0.717202380952381

In [53]:
scores['test_precision'].mean()

0.6755129316319821

In [57]:
from joblib import dump, load

In [58]:
dump(classifier_fitted, '../visualization/models/NaiveBayes_MultinomialNB.joblib') 

['../visualization/models/NaiveBayes_MultinomialNB.joblib']

In [59]:
dump(vectorizer_fitted, '../visualization/models/Vectorizer_NaiveBayes_MultinomialNB.joblib')

['../visualization/models/Vectorizer_NaiveBayes_MultinomialNB.joblib']

In [ ]:
vectorizer_fitted = vectorizer.fit(x.values)

In [67]:
vectorizer_fitted.transform(x_test.sample(1).values)

<1x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

array(['lugar onde me sinto desconfortavel bem vinda no propria casa pode porra desse mano'],
      dtype=object)